In [3]:
import cv2
import os
from ultralytics import YOLO
from collections import deque
from datetime import datetime
from PIL import Image
from IPython.display import display, clear_output
import time

# Load trained model
model = YOLO("runs/detect/intruder_yolov8n2/weights/best.pt")

# Detection parameters
target_class = "person"  # Change back to "backpack" for your project
min_confidence = 0.5
alert_hold_time = 5  # seconds of persistent detection required
fps = 10
video_frame_buffer = deque(maxlen=fps * alert_hold_time)
class_buffer = deque(maxlen=fps * alert_hold_time)

# Get save path (same directory as notebook)
save_dir = os.getcwd()

# Open camera
cap = cv2.VideoCapture(0)
frame_idx = 0
detection_start_time = None
alert_triggered = False

# Get video frame size
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

try:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_idx += 1
        current_frame_classes = []

        # Inference
        results = model(frame, verbose=False)[0]

        for box in results.boxes:
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])
            name = model.names[cls_id]

            if conf > 0.25:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{name} {conf:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

            if name == target_class and conf > min_confidence:
                current_frame_classes.append(name)

        # Logging detection
        if target_class in current_frame_classes:
            print(f"📌 Detected: {target_class} at frame {frame_idx}")
            if detection_start_time is None:
                detection_start_time = time.time()
        else:
            detection_start_time = None
            alert_triggered = False

        # Buffer management
        class_buffer.append(current_frame_classes)
        video_frame_buffer.append(frame.copy())

        # Check for persistent detection
        if detection_start_time:
            elapsed = time.time() - detection_start_time
            if elapsed >= alert_hold_time and not alert_triggered:
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                img_path = os.path.join(save_dir, f"alert_{target_class}_{timestamp}.jpg")
                video_path = os.path.join(save_dir, f"alert_{target_class}_{timestamp}.mp4")

                # Save alert image
                cv2.imwrite(img_path, frame)
                print(f"✅ Alert image saved: {img_path}")

                # Save alert video
                out = cv2.VideoWriter(video_path,
                                      cv2.VideoWriter_fourcc(*'mp4v'),
                                      fps,
                                      (frame_width, frame_height))
                for f in video_frame_buffer:
                    out.write(f)
                out.release()
                print(f"🎥 Alert video saved: {video_path}")

                alert_triggered = True
                detection_start_time = None
                video_frame_buffer.clear()

        # Display in notebook
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame_rgb)
        clear_output(wait=True)
        display(img)

except KeyboardInterrupt:
    print("⛔ Interrupted by user.")

finally:
    cap.release()
    cv2.destroyAllWindows()
    print("✅ Camera released.")


⛔ Interrupted by user.
✅ Camera released.
